In [14]:
import csv

# Define the input and output file paths
input_file = "/content/llm-human-readable__content_perfect6_part-00000-dba16e4b-d86f-4253-83c7-f31f2c533386-c000.csv"
output_file = "/content/cleaned_file.csv"

# Expected number of fields
expected_fields = 9

# Read the input file and clean the data
with open(input_file, 'r', encoding='utf-8') as infile, open(output_file, 'w', encoding='utf-8', newline='') as outfile:
    reader = csv.reader(infile)
    writer = csv.writer(outfile, quoting=csv.QUOTE_ALL)  # Quote all fields to handle commas

    # Write the header (assuming the first row is the header)
    header = next(reader)
    if len(header) != expected_fields:
        print(f"Header has {len(header)} fields, expected {expected_fields}: {header}")
        # Adjust the header if needed (e.g., remove extra columns)
        header = header[:expected_fields]
    writer.writerow(header)

    # Process each row
    for i, row in enumerate(reader, 2):  # Start counting from line 2 (after header)
        if len(row) != expected_fields:
            print(f"Line {i} has {len(row)} fields, expected {expected_fields}: {row}")
            # Fix the row by merging extra fields into the last expected field (e.g., County)
            if len(row) > expected_fields:
                # Merge extra fields into the last field (County)
                row[expected_fields-1] = ','.join(row[expected_fields-1:])  # Join extra fields into County
                row = row[:expected_fields]  # Keep only the first 9 fields
            elif len(row) < expected_fields:
                # Pad with empty strings if too few fields
                row.extend([''] * (expected_fields - len(row)))
        writer.writerow(row)

print(f"Cleaned data saved to {output_file}")

Streaming output truncated to the last 5000 lines.
Line 385318 has 10 fields, expected 9: ['AT&T Wireless', 'FWA', '1', '37', '0.027', 'CA', '93287', 'Woody', 'Kern County', 'The broadband carrier AT&T Wireless, offering FWA, serves 1 households out of 37 total households in Woody, Kern County, CA. This results in a carrier share of 2.7%, indicating a limited presence in the area.']
Line 385319 has 10 fields, expected 9: ['Verizon Wireless', 'FWA', '1', '68', '0.0147', 'MD', '21622', 'Church Creek', 'Dorchester County', 'The broadband carrier Verizon Wireless, offering FWA, serves 1 households out of 68 total households in Church Creek, Dorchester County, MD. This results in a carrier share of 1.47%, indicating a limited presence in the area.']
Line 385320 has 10 fields, expected 9: ['Brightspeed', 'FBB', '1', '15549', '0.0001', 'LA', '70560', 'New Iberia', 'Iberia Parish County', 'The broadband carrier Brightspeed, offering FBB, serves 1 households out of 15,549 total households in Ne

In [15]:
df = pd.read_csv("/content/cleaned_file.csv")

<ipython-input-15-5205ec166bd9>:1: DtypeWarning: Columns (2,3,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/cleaned_file.csv")


In [16]:
df.head()

,hh_bb_carrier,bb_type,distinct_hhid_count,total_hhids,carrier_share,State level,Zipcode level,City,County
0,Comcast,FBB,277860.0,281022.0,0.9887,NJ,8054,Mount Laurel,Burlington County
1,Comcast,FBB,195087.0,245469.0,0.7948,FL,33130,Miami,Miami-Dade County
2,Comcast,FBB,164147.0,179351.0,0.9152,PA,19107,Philadelphia,Philadelphia County
3,Comcast,FBB,152180.0,215616.0,0.7058,TX,77002,Houston,Harris County
4,Comcast,FBB,150933.0,152069.0,0.9925,MA,2238,Cambridge,Middlesex County


In [20]:
df.isnull().sum()

,0
hh_bb_carrier,0
bb_type,3
distinct_hhid_count,5
total_hhids,5
carrier_share,6
State level,8
Zipcode level,9
City,13
County,16


In [22]:
df=df.dropna()

In [24]:
df.shape

(422547, 9)

In [26]:
df.columns

Index(['hh_bb_carrier', 'bb_type', 'distinct_hhid_count', 'total_hhids',
       'carrier_share', 'State level', 'Zipcode level', 'City', 'County'],
      dtype='object')

In [27]:
import pandas as pd
import csv

# Define the state mapping
state_mapping = {
    'AL': 'Alabama', 'AK': 'Alaska', 'AZ': 'Arizona', 'AR': 'Arkansas', 'CA': 'California',
    'CO': 'Colorado', 'CT': 'Connecticut', 'DE': 'Delaware', 'FL': 'Florida', 'GA': 'Georgia',
    'HI': 'Hawaii', 'ID': 'Idaho', 'IL': 'Illinois', 'IN': 'Indiana', 'IA': 'Iowa',
    'KS': 'Kansas', 'KY': 'Kentucky', 'LA': 'Louisiana', 'ME': 'Maine', 'MD': 'Maryland',
    'MA': 'Massachusetts', 'MI': 'Michigan', 'MN': 'Minnesota', 'MS': 'Mississippi', 'MO': 'Missouri',
    'MT': 'Montana', 'NE': 'Nebraska', 'NV': 'Nevada', 'NH': 'New Hampshire', 'NJ': 'New Jersey',
    'NM': 'New Mexico', 'NY': 'New York', 'NC': 'North Carolina', 'ND': 'North Dakota', 'OH': 'Ohio',
    'OK': 'Oklahoma', 'OR': 'Oregon', 'PA': 'Pennsylvania', 'RI': 'Rhode Island', 'SC': 'South Carolina',
    'SD': 'South Dakota', 'TN': 'Tennessee', 'TX': 'Texas', 'UT': 'Utah', 'VT': 'Vermont',
    'VA': 'Virginia', 'WA': 'Washington', 'WV': 'West Virginia', 'WI': 'Wisconsin', 'WY': 'Wyoming',
    'DC': 'District of Columbia',
    'PR': 'Puerto Rico', 'GU': 'Guam', 'VI': 'U.S. Virgin Islands', 'AS': 'American Samoa',
    'MP': 'Northern Mariana Islands'
}

# Mapping for broadband types to their full names
bb_type_mapping = {
    'FBB': 'Fixed Broadband',
    'FWA': 'Fixed Wireless Access'
}

# Load the cleaned CSV file
df = pd.read_csv(
    '/content/cleaned_file.csv',
    quoting=csv.QUOTE_ALL,
    na_values=['']
)

# Clean the distinct_hhid_count column
def clean_distinct_hhid_count(value):
    try:
        return int(float(value))  # Convert to float first to handle strings like "277860.0", then to int
    except:
        return 0  # Default to 0 if not a number

df['distinct_hhid_count'] = df['distinct_hhid_count'].apply(clean_distinct_hhid_count)

# Clean the total_hhids column
def clean_total_hhids(value):
    try:
        return int(float(value))  # Convert to float first to handle strings like "281022.0", then to int
    except:
        return 0  # Default to 0 if not a number

df['total_hhids'] = df['total_hhids'].apply(clean_total_hhids)

# Clean the carrier_share column
def clean_carrier_share(value):
    try:
        return float(value)
    except:
        return 0.0  # Default to 0.0 if not a number

df['carrier_share'] = df['carrier_share'].apply(clean_carrier_share)

# Clean the State level column
def clean_state_level(value):
    value = str(value).strip().upper()
    if value not in state_mapping:
        return "Unknown"  # Default to "Unknown" if not a valid state code
    return value

df['State level'] = df['State level'].apply(clean_state_level)

# Clean other columns to ensure they are strings
df['hh_bb_carrier'] = df['hh_bb_carrier'].fillna('Unknown').astype(str)
df['bb_type'] = df['bb_type'].fillna('Unknown').astype(str)
df['Zipcode level'] = df['Zipcode level'].fillna('Unknown').astype(str)
df['City'] = df['City'].fillna('Unknown').astype(str)
df['County'] = df['County'].fillna('Unknown').astype(str)

# Function to generate the new summary based on the proposed template
def create_uniform_summary(row):
    try:
        # Extract values from the row using the correct column names
        zip_code = row['Zipcode level']  # e.g., 8054
        city = row['City']  # e.g., Mount Laurel
        county = row['County']  # e.g., Burlington
        state_code = row['State level']  # e.g., NJ
        state = state_mapping.get(state_code, state_code)  # Map state code to full name (e.g., NJ → New Jersey)
        carrier = row['hh_bb_carrier']  # e.g., Comcast
        bb_type = bb_type_mapping.get(row['bb_type'], row['bb_type'])  # Map FBB to Fixed Broadband
        distinct_hh = int(row['distinct_hhid_count'])  # Use the cleaned value directly
        total_hh = int(row['total_hhids'])  # Use the cleaned value directly
        carrier_share = float(row['carrier_share']) * 100  # Convert to percentage (e.g., 0.9887 → 98.87)

        # Format the summary using the proposed template
        summary = (f"In the zip code {zip_code} of {city}, {county} in the state of {state} ({state_code}) in USA, "
                   f"during 2024, July, {carrier} providing {bb_type} services served {distinct_hh:,} households "
                   f"out of a total of {total_hh:,} in the country, resulting in a market share of {carrier_share:.2f}%.")

        return summary
    except Exception as e:
        return f"Error processing row: {str(e)}"

# Create the Human_Readable_Summary column
df['Human_Readable_Summary'] = df.apply(create_uniform_summary, axis=1)

# Save the updated CSV
df.to_csv('/content/updated_file.csv', index=False)

# Display the updated summaries to verify
print(df[['Human_Readable_Summary']].head())

# Check for any errors in the summaries
errors = df[df['Human_Readable_Summary'].str.contains("Error processing row")]
if not errors.empty:
    print("\nRows with errors:")
    print(errors)
else:
    print("\nNo errors found in the summaries.")

<ipython-input-27-feb6361b3819>:28: DtypeWarning: Columns (2,3,4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(


                              Human_Readable_Summary
0  In the zip code 8054 of Mount Laurel, Burlingt...
1  In the zip code 33130 of Miami, Miami-Dade Cou...
2  In the zip code 19107 of Philadelphia, Philade...
3  In the zip code 77002 of Houston, Harris Count...
4  In the zip code 2238 of Cambridge, Middlesex C...

No errors found in the summaries.


In [28]:
df = pd.read_csv("/content/updated_file.csv")

<ipython-input-28-c27c7cf20540>:1: DtypeWarning: Columns (6) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("/content/updated_file.csv")


In [29]:
df.shape

(422563, 10)

In [30]:
df.isnull().sum()

,0
hh_bb_carrier,0
bb_type,0
distinct_hhid_count,0
total_hhids,0
carrier_share,6
State level,0
Zipcode level,0
City,0
County,0
Human_Readable_Summary,0
